# Songhay Publications: exploring the `MarkdownEntry`

In this Studio, a `MarkdownEntry` [📖 [GitHub](https://github.com/BryanWilhite/Songhay.Publications/blob/main/Songhay.Publications/Models/MarkdownEntry.cs)] is concerned with these things:

1. where is the text `Document` in the file system that contains Publication data
2. what is the Content of this text `Document`
3. what is the front matter of this text `Document`

In [Songhay Publications and the Concept of the Index](https://songhayblog.azurewebsites.net/entry/2020-12-24-songhay-publications-and-the-concept-of-the-index/), I introduce the `Segment`, `Document` and `Fragment`. These have been very generic types, historically defining a relational database schema. Relatively recently, I have introduced the `MarkdownEntry` as a kind of “data transfer object” (DTO) in order to formally recognize the existence of the Markdown file. Doing this will allow me to use tools like [Obsidian](https://obsidian.md/) in my Studio publication pipelines.

In [1]:
#!csharp

#r "nuget:Songhay.Publications"

Installed Packages Songhay.Publications, 8.0.3

## from the file system to `MarkdownEntry`

Let us see the `MarkdownEntry` in action with the concerns mentioned above:

In [2]:
#!csharp

using Songhay.Publications.Models;

MarkdownEntry mdEntry = new();

### the text `Document` in the file system that contains Publication data

There is a `*.md` file next to this Jupyter Notebook, called `songhay-publications-markdown-entry.md`:

In [3]:
#!csharp

using System.IO;

mdEntry.EntryFileInfo = new FileInfo("./songhay-publications-markdown-entry.md");

mdEntry.EntryFileInfo.Exists

True

### the Content of this text `Document`

Let us add content to this document:

In [4]:
#!csharp

mdEntry.Content =
"""
# My Document

Hello world! Markdown is _here_ to stay!
""";

### the front matter of this text `Document`

We can write the front matter of `MarkdownEntry` deliberately in the approximate ‘shape’ of our `Document` type mentioned above:

In [5]:
#!csharp

using System.Text.Json.Nodes;
using Songhay.Publications.Extensions;

mdEntry.FrontMatter["documentId"] = 101;
mdEntry.FrontMatter["title"] = "My Document";
mdEntry.FrontMatter["documentShortName"] = "my-doc";
mdEntry.FrontMatter["fileName"] = mdEntry.EntryFileInfo.Name;
mdEntry.FrontMatter["path"] = "./";
mdEntry.FrontMatter["templateId"] = null;
mdEntry.FrontMatter["segmentId"] = 1001;
mdEntry.FrontMatter["isRoot"] = true;
mdEntry.FrontMatter["isActive"] = true;
mdEntry.FrontMatter["sortOrdinal"] = null;
mdEntry.FrontMatter["clientId"] = "client-id-for-the-outside-world";
mdEntry.FrontMatter["tags"] = JsonNode.Parse("[ \"tag1\", \"tag2\" ]");
mdEntry.FrontMatter["rxExtract"] = "This is the document extract.";
mdEntry.FrontMatter["rxNextLink"] = null;
mdEntry.FrontMatter["rxPreviousLink"] = null;
mdEntry.FrontMatter["rxWrapperLink"] = null;

Our approximation of `Document` starts to fall apart after the `clientId` statement. This is done to establish Studio conventions for eleventy and Obsidian that are beyond the scope of these notes.

## saving the instance of `MarkdownEntry`

The most challenging part of preparing our instance of `MarkdownEntry` to save to disk is to convert the `JsonObject` into YAML. The `ToYaml` extension method [📖 [GitHub](https://github.com/BryanWilhite/Songhay.Publications/blob/f3a0f5b55a4de0795d7c0784e538119ce06657b4/Songhay.Publications/Extensions/JsonObjectExtensions.cs#L14C27-L14C33)] for `JsonObject` is used here:

In [6]:
#!csharp

using Microsoft.Extensions.Logging;
using Songhay;
using Songhay.Extensions;

ILogger logger = ILoggerUtility.AsInstanceOrNullLogger(null);

string yaml = mdEntry.FrontMatter.ToYaml(logger);

yaml

documentId: 101
title: My Document
documentShortName: my-doc
fileName: songhay-publications-markdown-entry.md
path: ./
templateId: 
segmentId: 1001
isRoot: true
isActive: true
sortOrdinal: 
clientId: client-id-for-the-outside-world
tags:
- tag1
- tag2
rxExtract: This is the document extract.
rxNextLink: 
rxPreviousLink: 
rxWrapperLink: 


With the `yaml` available, we can save:

In [7]:
#!csharp

using Songhay.Publications.Models;

string content =
$"""
{PublicationAppScalars.FrontMatterFence}
{yaml.Trim()}
{PublicationAppScalars.FrontMatterFence}

{mdEntry.Content}

""";

File.WriteAllText(mdEntry.EntryFileInfo.FullName, content);

…and verify the content of the document:

In [8]:
#!csharp

File.ReadAllText(mdEntry.EntryFileInfo.FullName)

---
documentId: 101
title: My Document
documentShortName: my-doc
fileName: songhay-publications-markdown-entry.md
path: ./
templateId: 
segmentId: 1001
isRoot: true
isActive: true
sortOrdinal: 
clientId: client-id-for-the-outside-world
tags:
- tag1
- tag2
rxExtract: This is the document extract.
rxNextLink: 
rxPreviousLink: 
rxWrapperLink:
---

# My Document

Hello world! Markdown is _here_ to stay!


## loading the `*.md` file into a conventional tuple

For the sake of flexibility and primacy, we can use the `ToJsonObjectAndAnyContent` extension method [📖 [GitHub](https://github.com/BryanWilhite/Songhay.Publications/blob/f3a0f5b55a4de0795d7c0784e538119ce06657b4/Songhay.Publications/Extensions/FileInfoExtensions.cs#L239C9-L239C34)] to reload our Markdown document:

In [9]:
#!csharp

using System.Text.Json;

var (frontMatter, content) = mdEntry.EntryFileInfo.ToJsonObjectAndAnyContent(logger);

frontMatter.ToJsonString(new JsonSerializerOptions { WriteIndented =  true })

{
  "documentId": "101",
  "title": "My Document",
  "documentShortName": "my-doc",
  "fileName": "songhay-publications-markdown-entry.md",
  "path": "./",
  "templateId": null,
  "segmentId": "1001",
  "isRoot": "true",
  "isActive": "true",
  "sortOrdinal": null,
  "clientId": "client-id-for-the-outside-world",
  "tags": [
    "tag1",
    "tag2"
  ],
  "rxExtract": "This is the document extract.",
  "rxNextLink": null,
  "rxPreviousLink": null,
  "rxWrapperLink": null
}

…we see that the YAML has been converted back to JSON and the `content` is just a simple string:

In [10]:
#!csharp

content


# My Document

Hello world! Markdown is _here_ to stay!

These items of the tuple can be used to make a new `MarkdownEntry` instance:

In [11]:
#!csharp

MarkdownEntry mdEntryCopy = new()
{
    FrontMatter = frontMatter,
    Content = content
};

## <!-- -->

[Bryan Wilhite is on LinkedIn](https://www.linkedin.com/in/wilhite)🇺🇸💼